# Notes on Phase Diversity

## Image model

The image amplitude at defocus $0$, $i_0$, is the convolution of the object amplitude $o$ and the broadband PSF at defocus $0$, $h_0$ (noise is neglected):

$$i_0(x, y, \lambda) = o(x, y, \lambda) * h_0(x, y, \lambda)$$

In the case of Euclid, $o$ is a star modeled as a point source (unknown position $(x_s, y_s)$) with known spectrum $S$:

$$
\begin{align*}
o(x_s, y_s, \lambda) &= S(\lambda) \\
o(x, y, \lambda) &= 0 \text{ for } (x, y) \neq (x_s, y_s)
\end{align*}
$$

## PSF model

The broadband PSF is the integral of the monochromatic PSFs over $\lambda$:

$$h_0 = \int_\lambda h_{\lambda,0} \, S(\lambda) \, \text{d}\lambda$$

Monochromatic PSFs are computed from the Fourier transform of the complex pupil function:

$$h_{\lambda,0}(x, y) = |\mathcal{F}^{-1}(P(u, v) \, \exp(-j2\pi/\lambda\cdot\Phi_0(u, v)))|^2$$

where:
* $P$ is the $1024 \times 1024$ pupil mask, values of which are reals in $(0, 1)$ for antialiasing at edges, but are mostly 0's and 1's;
* $\Phi_0$ is the WFE at defocus 0.

## WFE model

The WFE is decomposed into Zernike polynomials:

$$\Phi_0 = \sum_i \alpha_i Z_i$$

where the Zernike polynomial $Z_i$ is precomputed as a $1024 \times 1024$ matrix, and the Zernike coefficients $\alpha_i$ are the unknowns of interest.

## MTF model

The monochromatic MTF is useful to introduce non-optical PSFs, which are computed faster as multiplications in MTF space than as convolutions in PSF space.
The MTF $H$ is simply the complex Fourier transform of the PSF:

$$H_{\lambda,0} = \mathcal{F}(h_{\lambda,0})$$

## Phase diversity

Same equations hold at defocus $\delta$:

$$
\begin{align*}
h_{\lambda,\delta}(x, y) &= |\mathcal{F}^{-1}(P(u, v) \, \exp(-j2\pi/\lambda\cdot\Phi_\delta(u, v)))|^2 \\
\Phi_\delta &= \sum_i \beta_i Z_i
\end{align*}
$$

The defocus mechanism is known well-enough to assume that the $\beta_i$'s equal the $\alpha_i$'s except for $i = 4$ and $i = 11$.

## Error function

An error function $E$ is introduced, to be minimized:

$$
E = \sum_{(u,v)}\frac{\|i_0(u,v) - (a_0 \, h_0(u,v) + b_0)\|^2}{\sigma_0^2(u,v)}
+ \frac{\|i_\delta(u,v) - (a_\delta \, h_\delta(u,v) + b_\delta)\|^2}{\sigma_\delta^2(u,v)}
$$

with $a_0, b_0, a_\delta, b_\delta$ some unknown normalization coefficients.

The error function can be extended to more defocus values.

## Unknowns

From the above equations, the unknowns are:

- The position of the stars: $(x_s, y_s)$ for $s = 1..1000$;
- The Zernike coefficients: $\alpha_i$ for $i = 1..40$ as well as $\beta_4$ and $\beta_{11}$;
- The PSF normalization coefficients: $a_0, b_0, a_\delta, b_\delta$.

## Computation

Given:
* $L$ the number of wavelengths ($L \approx 40$);
* $M$ the number of Zernike polynomials ($M = 40$);
* $N$ the number of input images (star thumbnails, $N \in [100, 1000]$);

the error is computed $O(L \cdot M \cdot N)$ times per minimization iteration in order to build the Jacobian matrix (one-step derivatives).

The broadband PSF processing budget (including some non-aforementioned steps) is as follows:
* Inverse DFT ($1024^2$ real to complex without Hermitian symmetry) to estimate the $h_\lambda$'s from $P$ for each $\lambda$ (15%);
* Pixel-wise exponential ($1024^2$) for the same step (20%);
* Direct DFT ($1024^2$ real to complex with Hermitian symmetry) to estimate the $H_\lambda$'s from the $h_\lambda$'s (15%);
* Multiplications ($1024^2$) in Fourier space to introduce non-optical monochromatic MTFs (5%);
* Wavelength densification = interpolation ($512^2 \times 200$ points) along $\lambda$ (25%);
* Integration of monochromatic MTFs ($512^2 \times 200$ to $512^2$) (unit time not available, included in next step);
* Inverse DFT ($512^2$ complex with Hermitian symmetry to real) to get the broadband PSF from the broadband MTF (15%).

## Optimization opportunities

* Reduce the size of the output PSF (512 as of today, but could be smaller, see issue #1);
* Compute the exponential only where $P > 0$;
* Use pyFFTW efficiently;
* Compute DFT using FFTW single precision routines (see [FFTW
  documentation](https://www.fftw.org/fftw3_doc/Precision.html)).